# American: Brennan-Schwartz

In [1]:
import numpy as np

In [2]:
def tri_put(LL, DD, UU, B, s, K, N):
    L = np.empty(N+1)
    D = np.empty(N+1)
    U = np.empty(N+1)
    
    L[1:N+1] = np.copy(LL[1:N+1])
    D[1:N+1] = np.copy(DD[1:N+1])
    U[1:N+1] = np.copy(UU[1:N+1])
    
    for i in np.arange(1, N)[::-1]:
        Xmult = U[i]/D[i+1]
        D[i] = D[i] - Xmult*L[i+1]
        B[i] = B[i] - Xmult*B[i+1]
        
    Ic = 1
    B[Ic] = B[Ic]/D[Ic]
    
    while B[Ic] <= np.maximum(K - s[Ic], 0):
        B[Ic] = np.maximum(K - s[Ic], 0)
        B[Ic+1] = (B[Ic+1] - L[Ic+1]*B[Ic])/D[Ic+1]
        Ic += 1
    s_tau = s[Ic]

    for i in np.arange(Ic+1, N+1):
        B[i] = (B[i] - L[i]*B[i-1])/D[i]
    
    return s_tau

In [3]:
# set parameters
S0 = 100
K = 120
T = 0.5
lv = 0.3
rfr = 0.03
div = 0.01
SMIN = 10
SMAX = 410
N = 16000
M = 200

h = (SMAX - SMIN)/N
k = T/M

L = np.empty(N+1)
D = np.empty(N+1)
U = np.empty(N+1)

r = np.empty(N+1)
q = np.empty(N+1)

s_tau = np.empty(N+1)

s = SMIN + np.arange(0, N+1)*h
put = np.maximum(K - s, 0)

s_tau[0] = K

for j in np.arange(1, M+1):
    print(j)
    t_j = T - k*j
    r[j] = rfr
    q[j] = div
    for i in np.arange(1, N):
        # lv = 0.02*(s[i]/100)**(-1.)
        V = lv*lv
        A = (k/2)*(s[i]/h)*(s[i]/h)*V
        B = 0.5*k*s[i]*(r[j] - q[j])/h
        if i == 1:
            D[i] = 1 + r[j]*k + 2*B
            U[i] = -2*B
        elif i == N:
            L[i] = 2*B
            D[i] = 1 + r[j]*k - 2*B
        else:
            L[i] = -A + B 
            D[i] =  1 + r[j]*k + 2*A
            U[i] = -A - B
    s_tau[j] = tri_put(L, D, U, put, s, K, N-1);

for i in np.arange(1, N+1):
    if s[i] > S0:
        ir = i
        break
        
ps0 = (put[ir] - put[ir-1])*(S0 - s[ir-1])/h + put[ir-1]
print('Stock price: %f' % S0)
print('Put value: %f' % ps0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Stock price: 100.000000
Put value: 21.769465


In [4]:
f1 = open('premiums2.txt', 'w')
for i in range(1, N+1):
    f1.write("%g %g\n" % (s[i], put[i]))
f1.close()


f2 = open('free2.txt', 'w')
for j in range(0, M+1):
    f2.write("%g %g\n" % (k*j, s_tau[j]))
f2.close()